# Structured tool chat

The structured tool chat agent is capable of using multi-input tools.

Older agents are configured to specify an action input as a single string, but this agent can use the provided tools' `args_schema` to populate the action input.


In [1]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

## Initialize Tools

We will test the agent using a web browser

In [2]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install playwright

!playwright install

In [4]:
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()

## Use LCEL

We can first construct this agent using LangChain Expression Language

In [ ]:
from langchain import hub

In [19]:
prompt = hub.pull("hwchase17/react-multi-input-json")

In [ ]:
from langchain.tools.render import render_text_description_and_args

In [20]:
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [21]:
llm = ChatOpenAI(temperature=0)
llm_with_stop = llm.bind(stop=["Observation"])

In [ ]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

In [22]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
} | prompt | llm_with_stop | JSONAgentOutputParser()

In [ ]:
from langchain.agents import AgentExecutor

In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [24]:
response = await agent_executor.ainvoke({"input": "Browse to blog.langchain.dev and summarize the text, please."})
print(response['output'])



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Navigating to https://blog.langchain.dev returned status code 200Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```

LangChain LangChain Home GitHub Docs By LangChain Release Notes Write with Us Sign in Subscribe The official LangChain blog. Subscribe now Login Featured Posts Announcing LangChain Hub Using LangSmith to Support Fine-tuning Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications Sep 20 Peering Into the Soul of AI Decision-Making with LangSmith 10 min read Sep 20 LangChain + Docugami Webinar: Lessons from Deploying LLMs with LangSmith 3 min read Sep 18 TED AI Hackathon Kickoff (and projects we’d love to see) 2 min read Sep 12 How to Safely Query Enterprise Data with LangChain Agents + SQL + OpenAI + Gretel 6 min read Sep 12 OpaquePrompts x Lang

## Use off the shelf agent

In [5]:
llm = ChatOpenAI(temperature=0) # Also works well with Anthropic models
agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [7]:
response = await agent_chain.ainvoke({"input": "Browse to blog.langchain.dev and summarize the text, please."})
print(response['output'])



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Observation: Navigating to https://blog.langchain.dev returned status code 200
Thought:I have successfully navigated to the blog.langchain.dev website. Now I need to extract the text from the webpage to summarize it.
Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```
Observation: LangChain LangChain Home GitHub Docs By LangChain Release Notes Write with Us Sign in Subscribe The official LangChain blog. Subscribe now Login Featured Posts Announcing LangChain Hub Using LangSmith to Support Fine-tuning Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications Sep 20 Peering Into the Soul of AI Decision-Making with LangSmith 10 min read Sep 20 LangChain + Docugami Webinar: Lessons from Deploying LLMs with LangSmith 3 min read Sep 18 TED AI Hackathon Kickoff (and p